Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Installation and configuration
This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AML) Workspace.  You can use an existing workspace or create a new one.

In [1]:
import azureml.core
from azureml.core import Workspace, Datastore
from pathlib import Path
from AIHelpers.utilities import get_auth

## Prerequisites

If you have already completed the prerequisites and selected the correct Kernel for this notebook, the AML Python SDK is already installed. Let's check the AML SDK version.

In [2]:
print("AML SDK Version:", azureml.core.VERSION)

AML SDK Version: 1.0.76


## Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* Your subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If you have a workspace created already, you need to get your subscription and workspace information. You can find the values for those by visiting your workspace in the [Azure portal](http://portal.azure.com). If you don't have a workspace, the create workspace command in the next section will create a resource group and a workspace using the names you provide.

Replace the values in the following cell with your information. If you would like to use service principal authentication as described [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) make sure you provide the optional values as well. 

In [3]:
from dotenv import set_key, get_key, find_dotenv
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [8]:
# Azure resources
subscription_id = "0ca618d2-22a8-413a-96d0-0f1b531129c3"
resource_group = "dciborow-ado-test"  
workspace_name = "dciborowws"  
workspace_region = "westus"
image_name = (
    "dciborow_image"
)  # e.g. image_name = "{{cookiecutter.image_name}} (avoid underscore in names)"

sql_datastore_name=""
sql_server_name=""     # Name of Azure SQL server
sql_database_name=""   # Name of Azure SQL database
sql_username=""        # The username of the database user to access the database.
sql_password=""        # The password of the database user to access the database.

blob_datastore_name="" # Name of the Datastore  to workspace
datastore_rg = "agce-ai-mn-e2e-impact-score"
container_name="testdata"      # Name of Azure blob container
account_name="e2eimpactscore"        # Storage account name
account_key="Vd5PVsax4L9yXlfEOLw/+c+f3fK9/JU2ztFa7C43OCckCRYA5othHqeAz6x8Bs80ANX3IIPK90vBgEYNKcUieQ=="         # Storage account key

tenant_id = "YOUR_TENANT_ID" # Optional for service principal authentication
username = "YOUR_SERVICE_PRINCIPAL_APPLICATION_ID" # Optional for service principal authentication
password = "YOUR_SERVICE_PRINCIPAL_PASSWORD" # Optional for service principal authentication
storageConnString = "YOUR_STORAGE_CONNECTION_STRING"

Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [12]:
sql_datastore_name = "ado_sql_datastore" # Hard Code this for this example
blob_datastore_name="ado_blob_datastore"

#### Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.

This will fail when:
1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
2. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!  

In [9]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name, auth=cli_auth)

In [18]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=True,
    auth=get_auth(env_path),
    exist_ok=True,
)

TypeError: __init__() got an unexpected keyword argument 'username'

Let's check the details of the workspace.

In [10]:
ws.get_details()

{'id': '/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/dciborow-ado-test/providers/Microsoft.MachineLearningServices/workspaces/dciborowws',
 'name': 'dciborowws',
 'location': 'westus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Enterprise',
 'workspaceid': '3db1660f-e512-4152-a3ec-387abea8b04a',
 'description': '',
 'friendlyName': 'dciborowws',
 'creationTime': '2019-12-18T01:26:53.6336661+00:00',
 'keyVault': '/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourcegroups/dciborow-ado-test/providers/microsoft.keyvault/vaults/dciborowkeyvaultb9b3508b',
 'applicationInsights': '/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourcegroups/dciborow-ado-test/providers/microsoft.insights/components/dciborowinsights3fa1a864',
 'identityPrincipalId': 'ccc30575-90d5-4439-a8c0-59c168b285bc',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/0ca618d2-22a8-413a-

In [13]:
blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key,
                                                         resource_group=datastore_rg)

In [19]:
from azureml.core.dataset import Dataset
datastore_paths = [(blob_datastore, 'ai_impact_scores.csv')]

ai_impact_scores = Dataset.Tabular.from_delimited_files(path=datastore_paths)

ai_impact_scores.register(workspace=ws,
                          name="ai_impact_scores",
                          description = "ai subset of feedback items")

{
  "source": [
    "('ado_blob_datastore', 'ai_impact_scores.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "cfbfaa8a-e68b-438a-9a23-be134b15c3da",
    "name": "ai_impact_scores",
    "version": 1,
    "description": "ai subset of feedback items",
    "workspace": "Workspace.create(name='dciborowws', subscription_id='0ca618d2-22a8-413a-96d0-0f1b531129c3', resource_group='dciborow-ado-test')"
  }
}

In [17]:
ai_impact_scores.take(3).to_pandas_dataframe()

C:\Users\dcibo\Anaconda3\envs\MLHyperparameterTuning\lib\site-packages\azureml\dataprep\api\dataflow.py:689: UserWarning: Please install pyarrow>=0.11.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install azureml-dataprep[pandas].
  warnings.warn('Please install pyarrow>=0.11.0 for improved performance of to_pandas_dataframe. '


,Title,Priority,Mitigation Score String,IsBlocker,Count,Pri,Blocker,Mitigation,Score,LogScore,DCReview
0,Need support for deep learning in R programmin...,1,4,True,1,1,1,4,66.673125,1.823951,
1,Computer Vision API v2.0 Recognize Text (aka O...,2,5,True,3,0,1,5,50.004844,1.699012,-
2,Feature Gap: Need a service for Headline Detec...,2,5,True,3,0,1,5,50.004844,1.699012,


In [ ]:
# sql_datastore = Datastore.register_azure_sql_database(workspace=ws,
#                                                       datastore_name=sql_datastore_name,
#                                                       server_name=sql_server_name,
#                                                       database_name=sql_database_name,
#                                                       username=sql_username,
#                                                       password=sql_password)

Let's write the workspace configuration for the rest of the notebooks to connect to the workspace.

In [ ]:
ws.write_config()

You are now ready to move on to the [data preperation](01_DataPrep.ipynb) notebook.